In [1]:
!pip install 'git+https://github.com/paulgavrikov/visualkeras'

  Cloning https://github.com/paulgavrikov/visualkeras to /tmp/pip-req-build-8jok3e3s
  Running command git clone --filter=blob:none --quiet https://github.com/paulgavrikov/visualkeras /tmp/pip-req-build-8jok3e3s
  Resolved https://github.com/paulgavrikov/visualkeras to commit d7111d3471173c0b16853a6ce5e4cc645498380a
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 989.9/989.9 kB 2.6 MB/s eta 0:00:00a 0:00:01
  Created wheel for visualkeras: filename=visualkeras-0.0.2-py3-none-any.whl size=13449 sha256=3f1ded5c300d4ba4a58ea90df5dc859793c88b7acfdbb2caaa327748ed33d601
  Stored in directory: /tmp/pip-ephem-wheel-cache-sidz4vxd/wheels/b8/3b/cf/30a062e766188d5fe50abf3d461720943b7d09906e3d261a94
Successfully built visualkeras


In [2]:
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
import tensorflow.keras.optimizers
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread, imshow, imsave
import cv2
import numpy as np
import tensorflow
import visualkeras
from sklearn.metrics import plot_confusion_matrix# This Python 3 environment comes with many helpful analytics libraries installed

In [3]:
def load(file_path):
    image = imread(file_path)
    return image
def display(image,title="Image"):
    plt.figure(figsize=[10,10])
    channels=len(image.shape)
    if channels<3:
        plt.imshow(image,cmap='Greys_r');
    else:
        plt.imshow(image);      
    plt.title(title);plt.axis("off"); 
    
def check_label(file_path):
    if os.path.getsize(file_path) == 0:
        return 0
    else :
        return 1
def load_data(directory):
    my_counter_0=0
    my_counter_1=0
    sequence=["x/train","x/test","x/valid"]
    train_data=[]
    test_data=[]
    valid_data=[]
    
    valid_labels=[]
    test_labels=[]    
    train_labels=[]
    for x in sequence:
        directory_search=directory+x[1:]+'/images'
        for filename in os.listdir(directory_search):
            f = os.path.join(directory_search, filename)
            if os.path.isfile(f):
                my_image=load(f)
                resized = cv2.resize(my_image,(180,180), interpolation = cv2.INTER_AREA)
                if x == 'x/train' :
                    train_data.append(resized)
                elif x == 'x/test' :
                    test_data.append(resized)
                else:
                    valid_data.append(resized)
    
        directory_search=directory+x[1:]+'/labels'
        for filename in os.listdir(directory_search):
            f = os.path.join(directory_search, filename)
            if os.path.isfile(f):
                label=check_label(f)
                if label==0:
                    my_counter_0+=1
                elif label==1:
                    my_counter_1+=1
                if x == 'x/train' :
                    train_labels.append(label)
                elif x == 'x/test' :
                    test_labels.append(label)
                else:
                    valid_labels.append(label)
    print("Defected Pictures : ",my_counter_1)
    print("Non-Defected Pictures : ",my_counter_0)

    return train_data,test_data,valid_data,train_labels,test_labels,valid_labels

In [4]:
train_data,test_data,valid_data,train_labels,test_labels,valid_labels=load_data("/kaggle/input/dataset-final/Defect_Detection_FinalVersion")
np_train_data=np.array(train_data)
np_train_data = np_train_data.reshape((np_train_data.shape[0], 180,180, 3))
np_train_data = np_train_data.astype("float32") / 255.0
np_train_labels = np.asarray(train_labels).astype('float32').reshape((-1,1))

np_test_data=np.array(test_data)
np_test_data = np_test_data.reshape((np_test_data.shape[0], 180,180, 3))
np_test_data = np_test_data.astype("float32") / 255.0
np_test_labels = np.asarray(test_labels).astype('float32').reshape((-1,1))

np_valid_data=np.array(valid_data)
np_valid_data = np_valid_data.reshape((np_valid_data.shape[0], 180,180, 3))
np_valid_data = np_valid_data.astype("float32") / 255.0
np_valid_labels = np.asarray(valid_labels).astype('float32').reshape((-1,1))

Defected Pictures :  993
Non-Defected Pictures :  1109


In [5]:
class Resnet_50:    

    def build(imgRows, imgCols,numChannels,numClasses):
        resnet_model = Sequential()
        pretrained_resnet= tf.keras.applications.ResNet50V2(include_top=False,input_shape=(180,180,3),pooling='avg',classes=2,weights='imagenet',classifier_activation="relu")
        for each_layer in pretrained_resnet.layers:
                each_layer.trainable=False

        resnet_model.add(pretrained_resnet)
        resnet_model.add(Dense(512, activation='relu'))
        resnet_model.add(Dense(256, activation='relu'))
        resnet_model.add(Dense(1, activation='sigmoid'))
        return resnet_model

In [6]:
model = Resnet_50.build(numChannels=3, imgRows=640, imgCols=640,numClasses=2)
model.summary()

2023-01-08 12:58:47.875453: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


94683136/94668760 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 2048)              23564800  
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 24,745,473
Trainable params: 1,180,673
Non-trainable params: 23,564,800
_________________________________________________________________


In [7]:
visualkeras.layered_view(model, to_file='output.png') # write to disk

In [9]:
opt = keras.optimizers.Adam(learning_rate=0.02)
model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["accuracy"])
model.fit(np_train_data, np_train_labels, batch_size=64, epochs=40,verbose=1,validation_data=(np_valid_data, np_valid_labels))

Epoch 1/40
29/29 [==============================] - 89s 3s/step - loss: 0.6636 - accuracy: 0.6047 - val_loss: 0.7695 - val_accuracy: 0.4701
Epoch 2/40
29/29 [==============================] - 83s 3s/step - loss: 0.6243 - accuracy: 0.6436 - val_loss: 0.9799 - val_accuracy: 0.4627
Epoch 3/40
29/29 [==============================] - 84s 3s/step - loss: 0.6088 - accuracy: 0.6552 - val_loss: 0.7968 - val_accuracy: 0.5970
Epoch 4/40
29/29 [==============================] - 84s 3s/step - loss: 0.5971 - accuracy: 0.6634 - val_loss: 0.7859 - val_accuracy: 0.5448
Epoch 5/40
29/29 [==============================] - 83s 3s/step - loss: 0.5867 - accuracy: 0.6787 - val_loss: 1.0469 - val_accuracy: 0.4030
Epoch 6/40
29/29 [==============================] - 83s 3s/step - loss: 0.5881 - accuracy: 0.6787 - val_loss: 0.7357 - val_accuracy: 0.5522
Epoch 7/40
29/29 [==============================] - 82s 3s/step - loss: 0.5878 - accuracy: 0.6732 - val_loss: 0.7922 - val_accuracy: 0.5970
Epoch 8/40
29/29 [==

In [10]:
scores = model.evaluate(np_test_data, np_test_labels, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
model.save('Resnet_50')

accuracy: 47.92%


2023-01-08 14:08:49.930029: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
